In [1]:
import pandas as pd

In [2]:
pref = pd.read_csv("COVID-19-jp/nhk_news_covid19_prefectures_daily_data.csv")
pref["日付"] = pd.to_datetime(pref["日付"] )
print(len(pref))
pref.tail()

26790


,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計
26785,2021-08-03,47,沖縄県,467,25804,1,237
26786,2021-08-04,47,沖縄県,602,26406,0,237
26787,2021-08-05,47,沖縄県,648,27054,1,238
26788,2021-08-06,47,沖縄県,565,27619,1,239
26789,2021-08-07,47,沖縄県,548,28167,0,239


In [3]:
latest_date = pref["日付"].max().strftime("%Y-%m-%d")

In [4]:
all_jp = pref.groupby(["都道府県名"], as_index=False).sum()
del all_jp["各地の感染者数_累計"]
del all_jp["各地の死者数_累計"]
#all_jp = all_jp.rename(columns={"各地の感染者数合計":"各地の感染者数_1日ごとの発表数"})
all_jp = all_jp.rename(columns={"各地の感染者数_1日ごとの発表数":"各地の感染者数合計"})
all_jp = all_jp.rename(columns={"各地の死者数_1日ごとの発表数":"各地の死者数合計"})
all_jp.tail()

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計
42,静岡県,12540,12054,157
43,香川県,21090,2523,32
44,高知県,22230,2176,29
45,鳥取県,17670,970,3
46,鹿児島県,26220,4317,39


## 都道府県ごとの最新データを付与

In [5]:
# 直近１週間の感染者数
pref["直近1週間の感染者数"] = pref["各地の感染者数_1日ごとの発表数"].rolling(7).sum()
pref.tail(7)

,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計,直近1週間の感染者数
26783,2021-08-01,47,沖縄県,367,25128,0,236,2397.0
26784,2021-08-02,47,沖縄県,209,25337,0,236,2490.0
26785,2021-08-03,47,沖縄県,467,25804,1,237,2603.0
26786,2021-08-04,47,沖縄県,602,26406,0,237,2858.0
26787,2021-08-05,47,沖縄県,648,27054,1,238,3114.0
26788,2021-08-06,47,沖縄県,565,27619,1,239,3297.0
26789,2021-08-07,47,沖縄県,548,28167,0,239,3406.0


### 感染者数前日比

In [6]:
import numpy as np
pref["新規感染者数前日比"] = pref["各地の感染者数_1日ごとの発表数"].pct_change()
# 一番古い日付だけNANを設定
pref.loc[pref["日付"]==pref["日付"].min(),"新規感染者数前日比"] = np.nan
pref["直近1週間の新規感染者数前日比平均"] = pref["新規感染者数前日比"].rolling(7).mean()
pref["新規感染者数先週比"] = pref["各地の感染者数_1日ごとの発表数"].pct_change(periods=7)

pref.tail(10)

,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
26780,2021-07-29,47,沖縄県,392,23940,2,235,1617.0,0.129683,0.353052,1.562092
26781,2021-07-30,47,沖縄県,382,24322,1,236,1899.0,-0.025510,0.398894,2.820000
26782,2021-07-31,47,沖縄県,439,24761,0,236,2239.0,0.149215,0.421639,3.434343
26783,2021-08-01,47,沖縄県,367,25128,0,236,2397.0,-0.164009,0.239479,0.755981
26784,2021-08-02,47,沖縄県,209,25337,0,236,2490.0,-0.430518,0.241544,0.801724
26785,2021-08-03,47,沖縄県,467,25804,1,237,2603.0,1.234450,0.124791,0.319209
26786,2021-08-04,47,沖縄県,602,26406,0,237,2858.0,0.289079,0.168913,0.734870
26787,2021-08-05,47,沖縄県,648,27054,1,238,3114.0,0.076412,0.161303,0.653061
26788,2021-08-06,47,沖縄県,565,27619,1,239,3297.0,-0.128086,0.146649,0.479058
26789,2021-08-07,47,沖縄県,548,28167,0,239,3406.0,-0.030088,0.121034,0.248292


In [7]:
latest = pref.loc[pref["日付"]==pref["日付"].max()][["日付","都道府県名","各地の感染者数_1日ごとの発表数","各地の死者数_1日ごとの発表数","直近1週間の感染者数","新規感染者数前日比","直近1週間の新規感染者数前日比平均","新規感染者数先週比"]]
latest.head()

,日付,都道府県名,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
569,2021-08-07,北海道,347,1,2033.0,0.226148,0.063983,0.221831
1139,2021-08-07,青森県,30,0,152.0,0.111111,0.136413,0.363636
1709,2021-08-07,岩手県,39,0,143.0,0.392857,0.231400,0.857143
2279,2021-08-07,宮城県,106,0,557.0,-0.158730,0.175331,0.630769
2849,2021-08-07,秋田県,16,0,53.0,0.230769,0.517661,1.285714


In [8]:
all_jp = pd.merge(all_jp, latest, on="都道府県名")
all_jp.head()

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
0,三重県,13680,6209,114,2021-08-07,83,0,447.0,-0.144330,0.175957,0.886364
1,京都府,14820,19918,249,2021-08-07,271,0,1587.0,-0.062284,0.082846,0.361809
2,佐賀県,23370,2948,24,2021-08-07,38,0,201.0,0.117647,0.151781,0.652174
3,兵庫県,15960,46767,1320,2021-08-07,503,1,2780.0,0.058947,0.184869,0.528875
4,北海道,570,46457,1428,2021-08-07,347,1,2033.0,0.226148,0.063983,0.221831


## 人口データを付与

In [9]:
pop = pd.read_excel("COVID-19-jp/2001stjin.xls",header=3)[["Unnamed: 1","計"]]
pop.rename(columns={"Unnamed: 1":"都道府県名"},inplace=True)
pop.rename(columns={"計":"人口"},inplace=True)
pop = pop.loc[pop["都道府県名"]!="合計"].reset_index(drop=True)

In [10]:
all_jp = pd.merge(all_jp, pop, on="都道府県名", how="left")
all_jp.head()

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口
0,三重県,13680,6209,114,2021-08-07,83,0,447.0,-0.144330,0.175957,0.886364,1813859
1,京都府,14820,19918,249,2021-08-07,271,0,1587.0,-0.062284,0.082846,0.361809,2545899
2,佐賀県,23370,2948,24,2021-08-07,38,0,201.0,0.117647,0.151781,0.652174,823810
3,兵庫県,15960,46767,1320,2021-08-07,503,1,2780.0,0.058947,0.184869,0.528875,5549568
4,北海道,570,46457,1428,2021-08-07,347,1,2033.0,0.226148,0.063983,0.221831,5267762


## 人口当たりの感染者数

In [11]:
all_jp["１万人当たりの感染者数"] = all_jp["各地の感染者数合計"] / (all_jp["人口"] / 10000)
all_jp["10万人当たりの感染者数"] = all_jp["各地の感染者数合計"] / (all_jp["人口"] / 100000)
all_jp["直近1週間の10万人当たりの感染者数"] = all_jp["直近1週間の感染者数"] / (all_jp["人口"] / 100000)
all_jp.sort_values("都道府県コード")

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
4,北海道,570,46457,1428,2021-08-07,347,1,2033.0,0.226148,0.063983,0.221831,5267762,88.191152,881.911521,38.593239
41,青森県,1140,2850,32,2021-08-07,30,0,152.0,0.111111,0.136413,0.363636,1275783,22.339222,223.392223,11.914252
19,岩手県,1710,2145,47,2021-08-07,39,0,143.0,0.392857,0.231400,0.857143,1235517,17.361153,173.611533,11.574102
11,宮城県,2280,10463,91,2021-08-07,106,0,557.0,-0.158730,0.175331,0.630769,2292385,45.642420,456.424204,24.297838
36,秋田県,2850,1073,21,2021-08-07,16,0,53.0,0.230769,0.517661,1.285714,985416,10.888802,108.888023,5.378439
15,山形県,3420,2331,47,2021-08-07,37,0,190.0,-0.075000,0.221162,1.466667,1082296,21.537546,215.375461,17.555271
35,福島県,3990,6400,162,2021-08-07,99,0,644.0,0.253165,0.133288,0.207317,1881981,34.006720,340.067195,34.219262
38,茨城県,4560,14045,170,2021-08-07,256,0,1585.0,-0.095406,0.102227,0.488372,2921436,48.075672,480.756724,54.254141
27,栃木県,5130,9216,88,2021-08-07,164,0,892.0,0.188406,0.094861,-0.035294,1965516,46.888451,468.884507,45.382485
37,群馬県,5700,9684,155,2021-08-07,182,0,892.0,0.516667,0.312701,0.338235,1969439,49.171363,491.713630,45.292086


## 感染者数

### 平均値、中央値

In [12]:
all_jp["各地の感染者数合計"].describe()

count        47.000000
mean      21575.085106
std       42048.740659
min         795.000000
25%        2899.000000
50%        6209.000000
75%       13389.000000
max      245219.000000
Name: 各地の感染者数合計, dtype: float64

### TOP 10

In [13]:
all_jp.sort_values("各地の感染者数合計", ascending=False).reset_index(drop=True).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
0,東京都,7410,245219,2310,2021-08-07,4566,4,27251.0,0.011296,0.057339,0.125185,13834925,177.246353,1772.463530,196.972517
1,大阪府,15390,121294,2731,2021-08-07,1123,3,7159.0,-0.142748,0.121867,0.079808,8849635,137.061020,1370.610200,80.895992
2,神奈川県,7980,94893,1005,2021-08-07,1893,1,11545.0,-0.090778,0.047281,0.198101,9209442,103.038816,1030.388160,125.360472
3,埼玉県,6270,64756,854,2021-08-07,1449,1,7816.0,0.187705,0.063076,0.398649,7390054,87.625882,876.258820,105.763774
4,愛知県,13110,56316,1013,2021-08-07,459,1,2222.0,0.186047,0.124432,0.599303,7575530,74.339353,743.393532,29.331281
5,千葉県,6840,55410,736,2021-08-07,1075,0,6248.0,0.017029,0.046133,0.357323,6319772,87.677214,876.772137,98.864326
6,兵庫県,15960,46767,1320,2021-08-07,503,1,2780.0,0.058947,0.184869,0.528875,5549568,84.271424,842.714244,50.093989
7,北海道,570,46457,1428,2021-08-07,347,1,2033.0,0.226148,0.063983,0.221831,5267762,88.191152,881.911521,38.593239
8,福岡県,22800,43722,537,2021-08-07,742,0,4352.0,-0.114558,0.084635,0.472222,5129841,85.230712,852.307118,84.836937
9,沖縄県,26790,28167,239,2021-08-07,548,0,3406.0,-0.030088,0.121034,0.248292,1481547,190.118842,1901.188420,229.894833


## １万人あたりの感染者数

### 平均値、中央値

In [14]:
all_jp["１万人当たりの感染者数"].describe()

count     47.000000
mean      50.660226
std       39.145651
min       10.888802
25%       25.338485
50%       34.230886
75%       59.885962
max      190.118842
Name: １万人当たりの感染者数, dtype: float64

## 直近１週間の１０万人あたりの感染者数

In [15]:
all_jp.sort_values(["直近1週間の10万人当たりの感染者数"],ascending=False).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
28,沖縄県,26790,28167,239,2021-08-07,548,0,3406.0,-0.030088,0.121034,0.248292,1481547,190.118842,1901.188420,229.894833
26,東京都,7410,245219,2310,2021-08-07,4566,4,27251.0,0.011296,0.057339,0.125185,13834925,177.246353,1772.463530,196.972517
32,神奈川県,7980,94893,1005,2021-08-07,1893,1,11545.0,-0.090778,0.047281,0.198101,9209442,103.038816,1030.388160,125.360472
7,埼玉県,6270,64756,854,2021-08-07,1449,1,7816.0,0.187705,0.063076,0.398649,7390054,87.625882,876.258820,105.763774
5,千葉県,6840,55410,736,2021-08-07,1075,0,6248.0,0.017029,0.046133,0.357323,6319772,87.677214,876.772137,98.864326
34,福岡県,22800,43722,537,2021-08-07,742,0,4352.0,-0.114558,0.084635,0.472222,5129841,85.230712,852.307118,84.836937
9,大阪府,15390,121294,2731,2021-08-07,1123,3,7159.0,-0.142748,0.121867,0.079808,8849635,137.061020,1370.610200,80.895992
1,京都府,14820,19918,249,2021-08-07,271,0,1587.0,-0.062284,0.082846,0.361809,2545899,78.235625,782.356252,62.335544
38,茨城県,4560,14045,170,2021-08-07,256,0,1585.0,-0.095406,0.102227,0.488372,2921436,48.075672,480.756724,54.254141
31,石川県,9690,5677,118,2021-08-07,82,0,573.0,0.051282,0.033696,-0.088889,1139612,49.815200,498.152003,50.280271


## 直近１週間の新規感染者 増加速度

## 増加速度 統計情報

In [16]:
all_jp["直近1週間の新規感染者数前日比平均"].describe()

count    47.000000
mean      0.170045
std       0.126441
min       0.033696
25%       0.087295
50%       0.141332
75%       0.175644
max       0.571980
Name: 直近1週間の新規感染者数前日比平均, dtype: float64

### WORST 10(増加速度の早い順)

In [17]:
all_jp.sort_values("直近1週間の新規感染者数前日比平均",ascending=False).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
14,山口県,19950,3420,80,2021-08-07,31,0,123.0,0.476190,0.571980,0.823529,1369882,24.965654,249.656540,8.978876
36,秋田県,2850,1073,21,2021-08-07,16,0,53.0,0.230769,0.517661,1.285714,985416,10.888802,108.888023,5.378439
39,長崎県,23940,3730,69,2021-08-07,55,0,269.0,0.222222,0.511711,2.235294,1350769,27.613900,276.138999,19.914582
23,愛媛県,21660,3140,77,2021-08-07,56,0,215.0,0.400000,0.425473,1.947368,1369131,22.934255,229.342554,15.703391
17,岐阜県,11970,9897,189,2021-08-07,67,1,308.0,0.488889,0.359884,2.722222,2032490,48.693967,486.939665,15.153826
37,群馬県,5700,9684,155,2021-08-07,182,0,892.0,0.516667,0.312701,0.338235,1969439,49.171363,491.713630,45.292086
46,鹿児島県,26220,4317,39,2021-08-07,71,0,349.0,-0.183908,0.290028,2.380952,1630146,26.482291,264.822905,21.409125
12,宮崎県,25650,3431,27,2021-08-07,40,0,203.0,0.212121,0.237606,1.857143,1095903,31.307515,313.075154,18.523537
19,岩手県,1710,2145,47,2021-08-07,39,0,143.0,0.392857,0.231400,0.857143,1235517,17.361153,173.611533,11.574102
15,山形県,3420,2331,47,2021-08-07,37,0,190.0,-0.075000,0.221162,1.466667,1082296,21.537546,215.375461,17.555271


### BEST 10(増加速度の遅い順／減少速度の速い順)

In [18]:
all_jp.sort_values("直近1週間の新規感染者数前日比平均",ascending=True).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
31,石川県,9690,5677,118,2021-08-07,82,0,573.0,0.051282,0.033696,-0.088889,1139612,49.815200,498.152003,50.280271
6,和歌山県,17100,3208,49,2021-08-07,37,0,237.0,0.233333,0.037698,0.088235,954258,33.617743,336.177428,24.836051
40,長野県,11400,5647,94,2021-08-07,49,0,353.0,-0.183333,0.039618,0.195122,2087307,27.053998,270.539983,16.911743
5,千葉県,6840,55410,736,2021-08-07,1075,0,6248.0,0.017029,0.046133,0.357323,6319772,87.677214,876.772137,98.864326
32,神奈川県,7980,94893,1005,2021-08-07,1893,1,11545.0,-0.090778,0.047281,0.198101,9209442,103.038816,1030.388160,125.360472
26,東京都,7410,245219,2310,2021-08-07,4566,4,27251.0,0.011296,0.057339,0.125185,13834925,177.246353,1772.463530,196.972517
33,福井県,10260,1825,36,2021-08-07,32,0,219.0,-0.085714,0.061264,0.103448,780053,23.395846,233.958462,28.075015
7,埼玉県,6270,64756,854,2021-08-07,1449,1,7816.0,0.187705,0.063076,0.398649,7390054,87.625882,876.258820,105.763774
4,北海道,570,46457,1428,2021-08-07,347,1,2033.0,0.226148,0.063983,0.221831,5267762,88.191152,881.911521,38.593239
43,香川県,21090,2523,32,2021-08-07,59,0,251.0,0.439024,0.069235,0.372093,981280,25.711316,257.113158,25.578836


## グラフ化

In [19]:
import matplotlib.pyplot as plt
%matplotlib notebook
#%matplotlib inline

In [20]:
all_jp_desc = all_jp.sort_values(["各地の感染者数合計"],ascending=False)
all_jp_desc_10k = all_jp.sort_values(["１万人当たりの感染者数"],ascending=False)
all_jp_desc_100k = all_jp.sort_values(["直近1週間の10万人当たりの感染者数"],ascending=False)

In [21]:
fig = plt.figure(figsize=(9.5,5))
plt.subplot(1,2,1) # (rows, columns, panel number)
plt.title("各地の新規感染者数累計")
plt.yticks(fontsize=8)
plt.grid(axis="x")
chart_data = all_jp.sort_values(["各地の感染者数合計"])
plt.barh(chart_data["都道府県名"],chart_data["各地の感染者数合計"],align="center")

plt.subplot(1,2,2) # (rows, columns, panel number)
plt.title("1万人当たりの新規感染者数累計")
plt.yticks(fontsize=8)
plt.grid(axis="x")
chart_data = all_jp.sort_values(["１万人当たりの感染者数"])
plt.barh(chart_data["都道府県名"], chart_data["１万人当たりの感染者数"])

fig.text(0,0,"※新型コロナ関連の情報提供:NHK ※人口の情報提供:e-Stat",fontsize=8)
fig.text(0,0.98,"※{}時点".format(latest_date),fontsize=9,va="top")

fig.tight_layout()

<IPython.core.display.Javascript object>

In [22]:
fig = plt.figure(figsize=(9.5,10))

#plt.style.use("ggplot")
### 最新の各地の新規感染者数
plt.subplot(2,2,1) # (rows, columns, panel number)
plt.title("{}の各地の新規感染者数".format(latest_date))
chart_data = all_jp.sort_values(["各地の感染者数_1日ごとの発表数"])
plt.barh(chart_data["都道府県名"], chart_data["各地の感染者数_1日ごとの発表数"])
for i,v in enumerate(chart_data["各地の感染者数_1日ごとの発表数"]):
    plt.text(v,i,"{:,}".format(v),va="center",fontsize=7)
plt.yticks(fontsize=8)
plt.grid(axis="x")

### 最新の各地の死者数
plt.subplot(2,2,2) # (rows, columns, panel number)
plt.title("{}の各地の新規死者数".format(latest_date))
chart_data = all_jp.sort_values(["各地の死者数_1日ごとの発表数"])
plt.barh(chart_data["都道府県名"], chart_data["各地の死者数_1日ごとの発表数"])
plt.yticks(fontsize=8)
plt.grid(axis="x")



### 直近1週間の10万人当たりの新規感染者数
plt.subplot(2,2,3) # (rows, columns, panel number)
plt.title("直近1週間の10万人当たりの新規感染者数")
chart_data = all_jp.sort_values(["直近1週間の10万人当たりの感染者数"])
plt.barh(chart_data["都道府県名"], chart_data["直近1週間の10万人当たりの感染者数"])
for i,v in enumerate(chart_data["直近1週間の10万人当たりの感染者数"]):
    plt.text(v,i,"{:,}".format(int(v)),va="center",fontsize=7)
plt.plot([15,15],[0,47],"--",lw=0.5,color="black")
plt.text(15,0,"Stage3",ha="center",va="top",fontsize=8)
plt.plot([25,25],[0,47],"--",lw=0.5,color="black")
plt.text(25,0,"Stage4",ha="center",va="top",fontsize=8)
plt.yticks(fontsize=8)
plt.grid(axis="x")

### 新規感染者数の増加速度（新規感染者数前週同曜日比）
plt.subplot(2,2,4) # (rows, columns, panel number)
plt.title("新規感染者数前週同曜日比")
chart_data = all_jp.sort_values("新規感染者数先週比",na_position="first")
plt.barh(chart_data["都道府県名"], chart_data["新規感染者数先週比"])
plt.yticks(fontsize=8)
plt.grid(axis="x")

fig.text(0,0,"※新型コロナ関連の情報提供:NHK ※人口の情報提供:e-Stat",fontsize=8)
fig.text(0,0.98,"※{}時点".format(latest_date),fontsize=9,va="top")

fig.tight_layout()

<IPython.core.display.Javascript object>

In [23]:
fig.savefig("out/covid-domestic.png")

## 度数分布表

In [24]:
import numpy as np
data = np.array(all_jp["各地の感染者数合計"])
 
# ヒストグラム
hist, bin_edges = np.histogram(data, bins=8)

print(hist)
print(bin_edges)

[38  5  1  2  0  0  0  1]
[   795.  31348.  61901.  92454. 123007. 153560. 184113. 214666. 245219.]


## ヒストグラム

In [25]:
fig = plt.figure(figsize=(10,3))
plt.hist(all_jp_desc["各地の感染者数合計"],bins=[0,20000,40000,80000,100000,120000,140000,160000])

<IPython.core.display.Javascript object>

(array([37.,  1.,  6.,  1.,  0.,  1.,  0.]),
 array([     0,  20000,  40000,  80000, 100000, 120000, 140000, 160000]),
 <a list of 7 Patch objects>)

In [26]:
fig = plt.figure(figsize=(10,3))
plt.hist(all_jp_desc["10万人当たりの感染者数"],bins=[0,200,400,600,800,1000,1200])


<IPython.core.display.Javascript object>

(array([ 4., 21., 10.,  3.,  5.,  1.]),
 array([   0,  200,  400,  600,  800, 1000, 1200]),
 <a list of 6 Patch objects>)

In [27]:
pref[pref["都道府県名"]=="山口県"].tail(10)


,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
19940,2021-07-29,35,山口県,16,3253,0,80,49.0,0.600000,NaN,3.000000
19941,2021-07-30,35,山口県,27,3280,0,80,76.0,0.687500,NaN,inf
19942,2021-07-31,35,山口県,17,3297,0,80,91.0,-0.370370,0.459136,7.500000
19943,2021-08-01,35,山口県,9,3306,0,80,95.0,-0.470588,0.177624,0.800000
19944,2021-08-02,35,山口県,3,3309,0,80,91.0,-0.666667,0.025243,-0.571429
19945,2021-08-03,35,山口県,15,3324,0,80,97.0,4.000000,0.555855,0.666667
19946,2021-08-04,35,山口県,27,3351,0,80,114.0,0.800000,0.654268,1.700000
19947,2021-08-05,35,山口県,17,3368,0,80,115.0,-0.370370,0.515643,0.062500
19948,2021-08-06,35,山口県,21,3389,0,80,109.0,0.235294,0.451043,-0.222222
19949,2021-08-07,35,山口県,31,3420,0,80,123.0,0.476190,0.571980,0.823529
